In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:

def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = Y[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr


def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))

def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [3]:
iris_data = pd.read_csv("../Iris.csv")
iris_data['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

In [4]:
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
x = pd.DataFrame(iris_data, columns=columns)
x = normalize(x.values)

In [5]:
columns = ['Species']
y = pd.DataFrame(iris_data, columns=columns)
y = y.values
y = y.flatten()
y = to_one_hot(y)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)  


In [7]:
def generate_weights(count, input_count, output_count, internal_count):
    np.random.seed(0)
    result = [2*np.random.random((input_count, internal_count)) - 1]
    for i in range(count-2):
        result.append(2*np.random.random((internal_count, internal_count)) - 1)
    result.append(2*np.random.random((internal_count, output_count)) - 1)
    return result

In [8]:
def generate_layers(input_layer, weigths, use_relu_for_internal=False):
    result = [input_layer]
    if use_relu_for_internal:
        for w in weigths[:-1]:
            result.append(np.maximum(0,np.dot(result[-1], w)))
        result.append(sigmoid(np.dot(result[-1], weigths[-1]))) 
        return result                      

    for w in weigths:
        result.append(sigmoid(np.dot(result[-1], w)))
    return result

In [9]:
def correct_weight(y_train, layers,weight, lerning_rate):
    delta = (y_train - layers[-1]) * sigmoid_deriv(layers[-1])
    error = delta
    for i in range(len(layers)-2,-1,-1):
        layer_error = delta.dot(weight[i].T)
        weight[i] += layers[i].T.dot(delta) * lerning_rate
        delta = layer_error * sigmoid_deriv(layers[i])
  
    return error

__Вариант 1. Увеличение количества слоев__

In [10]:
weights = generate_weights(30,4,3,5)

n = 0.01 
errors = []

for i in range(100000):
    layers = generate_layers(X_train, weights)
    error = correct_weight(y_train, layers, weights, n)
    error = np.mean(np.abs(error))
    accuracy = (1 - error) * 100


print("Точность нейронной сети " + str(round(accuracy,2)) + "%")

Точность нейронной сети 89.25%


__Вариант 2. Увеличение количества нейронов__

In [11]:
weights = generate_weights(2,4,3,200)

n = 0.1 
errors = []

for i in range(100000):
    layers = generate_layers(X_train, weights)
    error = correct_weight(y_train, layers, weights, n)
    error = np.mean(np.abs(error))
    accuracy = (1 - error) * 100


print("Точность нейронной сети " + str(round(accuracy,2)) + "%")

Точность нейронной сети 99.65%


__Вариант 3. Уменьшение скорости обучения__

In [12]:
weights = generate_weights(2,4,3,5)

n = 0.000001 
errors = []

for i in range(100000):
    layers = generate_layers(X_train, weights)
    error = correct_weight(y_train, layers, weights, n)
    error = np.mean(np.abs(error))
    accuracy = (1 - error) * 100
    
print("Точность нейронной сети " + str(round(accuracy,2)) + "%")

Точность нейронной сети 88.81%


__Вариант 4. ReLu__

In [13]:
weights = generate_weights(2,4,3,5)

n = 0.01 
errors = []

for i in range(100000):
    layers = generate_layers(X_train, weights, True)
    error = correct_weight(y_train, layers, weights, n)
    error = np.mean(np.abs(error))
    accuracy = (1 - error) * 100
    
print("Точность нейронной сети " + str(round(accuracy,2)) + "%")

Точность нейронной сети 94.66%


__Выводы__

1. Бездумное увеличение слоев не всегда ведёт к повышению точности.   
2. Увеличение количества нейронов повышает точность
3. Если сильно уменьшить скорость обучения, нейронная сеть может не успеть обучится за отведенное количество итераций.   